In [1]:
import os
try:
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('drive/MyDrive/Colab Notebooks/PasswordGuessing')
except Exception as e:
  print(e)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
from google.colab import drive

print(f"Found GPU at: {tf.test.gpu_device_name()}.\nTensorFlow version: {tf.__version__}")

Found GPU at: /device:GPU:0.
TensorFlow version: 2.9.2


<ipython-input-2-195ce2e441a6>:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
path_to_file ="rockyou-train.txt" 
text = open(path_to_file, 'rb').read().decode(encoding='latin-1')
print ("The dataset has", len(text), "characters in total.")

The dataset has 172941082 characters in total.


In [4]:
unique=set()
for val in text:
  unique.add(val)

In [5]:
data_vocab={}
i=0
for val in unique:
  data_vocab[val]=i
  i=i+1


In [45]:
index_vocab={}
for key in data_vocab:
  index_vocab.update({data_vocab[key]:key})

In [6]:
import torch
from torch import nn

import math
import matplotlib.pyplot as plt

In [7]:
temp_data=text.split('\n')
data=[]
count=0
for val in temp_data:
  if len(val)==8 and count<5000:
     temp=[]
     for i in val:
       temp.append(data_vocab[i])
     data.append((torch.tensor(temp),torch.tensor(0)))
     count=count+1

In [9]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(
    data[0:4000], batch_size=batch_size, shuffle=True
)

In [10]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(8, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        output = self.model(x)
        return output
discriminator = Discriminator()

In [11]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 8),
        )

    def forward(self, x):
        output = self.model(x)
        return output

generator = Generator()

In [12]:
lr = 0.001
num_epochs = 300
loss_function = nn.BCELoss()

In [13]:
optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

In [81]:
for epoch in range(num_epochs):
    for n, (real_samples, _) in enumerate(train_loader):
        # Data for training the discriminator
        real_samples_labels = torch.ones((batch_size, 1))
        latent_space_samples = torch.randint(0,205,(batch_size, 8)).float()
        generated_samples = generator(latent_space_samples)
        generated_samples_labels = torch.zeros((batch_size, 1))
        all_samples = torch.cat((real_samples, generated_samples))
        all_samples_labels = torch.cat(
            (real_samples_labels, generated_samples_labels)
        )

        # Training the discriminator
        discriminator.zero_grad()
        output_discriminator = discriminator(all_samples)
        loss_discriminator = loss_function(output_discriminator, all_samples_labels)
        loss_discriminator.backward()
        optimizer_discriminator.step()

        # Data for training the generator
        latent_space_samples = torch.randint(0,205,(batch_size, 8)).float()

        # Training the generator
        generator.zero_grad()
        generated_samples = generator(latent_space_samples)
        output_discriminator_generated = discriminator(generated_samples)
        loss_generator = loss_function(output_discriminator_generated, real_samples_labels)
        loss_generator.backward()
        optimizer_generator.step()

        # Show loss
        if epoch % 10 == 0 and n == batch_size - 1:
            print(f"Epoch: {epoch} Loss D.: {loss_discriminator}")
            print(f"Epoch: {epoch} Loss G.: {loss_generator}")

Epoch: 0 Loss D.: 0.6050512194633484
Epoch: 0 Loss G.: 0.9099114537239075
Epoch: 10 Loss D.: 0.5674253702163696
Epoch: 10 Loss G.: 1.0046162605285645
Epoch: 20 Loss D.: 0.6670973300933838
Epoch: 20 Loss G.: 0.9398168921470642
Epoch: 30 Loss D.: 0.5668880939483643
Epoch: 30 Loss G.: 0.8893648982048035
Epoch: 40 Loss D.: 0.6694773435592651
Epoch: 40 Loss G.: 0.7776889204978943
Epoch: 50 Loss D.: 0.6489283442497253
Epoch: 50 Loss G.: 0.8023090958595276
Epoch: 60 Loss D.: 0.5935755968093872
Epoch: 60 Loss G.: 0.7705178260803223
Epoch: 70 Loss D.: 0.7120296955108643
Epoch: 70 Loss G.: 0.8501303195953369
Epoch: 80 Loss D.: 0.6790285706520081
Epoch: 80 Loss G.: 0.7418250441551208
Epoch: 90 Loss D.: 0.579420804977417
Epoch: 90 Loss G.: 0.752191424369812
Epoch: 100 Loss D.: 0.6318315267562866
Epoch: 100 Loss G.: 0.8673273324966431
Epoch: 110 Loss D.: 0.6341710090637207
Epoch: 110 Loss G.: 0.8154998421669006
Epoch: 120 Loss D.: 0.686642587184906
Epoch: 120 Loss G.: 0.8458014726638794
Epoch: 130 

In [82]:
def get_string(array):
  temp=''
  for val in array.tolist():
    temp=temp+index_vocab[val]
  return temp

In [83]:
def get_predicted_string(prediction):
  temp=''
  for i in prediction.tolist():
    if(i<0):
      temp=temp+index_vocab[0]
    elif(i>204):
      temp=temp+index_vocab[204]
    else:
      if(i-(i//1)<=0.5):
        temp=temp+index_vocab[i//1]
      else:
        temp=temp+index_vocab[i//1+1]
  return temp

In [84]:
for val in data[0:1000]:
  prediction = generator(val[0].float())
  print(get_predicted_string(prediction))


:wµb!ç
Î gª!
Ñý%f£l
{½:m¿
££8£ßf£
°l£g£gì
£4££a£7Ë
Ey7ÐíQn
ïÂ~~9kÆ[
Ï/-£°Aý
#×dða&o
`m®÷£8
%ý°½úß
¼¸¢W»&e¦
£af]øb£±
£»<:ø
<!ß Z®
|»BéYú£
®¾°t-ÏúÏ
ýÐÑæ m£±
wi@¯6¿
?mG§«~PË
£#££¯£££
êø´N££â0
çü@ú,
OÖº%Ix
£
M£H4££
¶_j©Å×£<
£M(y ££É
#¦x¯m'
M[¤, ã¥
}nN:ühcY
Ï%£×B7°H
©<vI cj!
ß£v!Å
éjý+£w
%#ù12£££
½u w¼ð£
®Äàiâ`7
}S,Â$);J
ê{°»Zé¥²
JK)]¥££q
Ã&o4[×Ñ
bJ8&Ý]®
±év¦f±i
°~F×\®£©
-D®T8O£²
¦Ò££<=`©
8£º£<££
üK´K¼_d
U9K£
4ZÎNg£n
ö¶àL½Ñ
£Ù££T°£É
V)ý`x£&
¹In'Ïä©$
T:¸D^£d
¨ 5á­£©
À¯[.¬£y
'\ryá|A
êo_&ðf|
£'£££££
¥T=C4
>.¯®(©
ùvmý:+ª,
F-X¸Q_Z,
¸8§Ae¼
Î7!sBwÃd
Ã©á4X¬^
b* FßH£c
c³Öu°I
MGFR÷n
^&M«_¨ 
iö¼¢°o
ýË$W¢
ªO££÷££
çéâC§£2
£U£í£xm

£°Æ}`x£]
:0ß¼£å
 ££'££
*¸,_±ö£\
UmïbB£ 
¹Ry4
£gY£X?ñ
£úÈ<ßãr
¢vbÝ:Ï
äKEi½gÙÑ
££©m£]£
BÒ£Òr%\±
¸ú+Â%d0
*>öX-m
ú£YgZ£ñ
ú
¹^Ï_P
WAZc£
f££ªn[y
ÝÐÐs/¾XË
££rv£Ï
°#Cº£ê
j]£££Ð£·
=¶#Lªvd
@:é1£5£
ïÅæX®,
×-£ÒP£á
¨Ï£q£Ö££
¯ï¶ö×<
£çOÖmg£+
!ÄS½c®
CLIÃW¢D
or£ØB#[G
bàzTº£¿
¯